# Which GPU to use

In [1]:
multiGPU = False
whichGPU = 1
 
# Select which GPU to use
if(multiGPU):
    from keras.utils.training_utils import multi_gpu_model
else:
    import os
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
    # The GPU id to use, usually either "0" or "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(whichGPU)
    
# # Do other imports now...

# Load all the functions

In [2]:
%run -i 'arena.py'

Using TensorFlow backend.


# General Parameters

In [3]:
import math

# What data to use
tableBase = '3PKk'
convertStates = False

# Interactive (just in general if one is asked for confirmations, set to False if on autopilot over night f.x.)
askForConfirmation = False

# NN parameters
filters = [16,32,32,64,128,128,128]
filterShape = [2,2,2,2,2,2,2]
batch_size = 256
optimizer = 'Adadelta'
useBatchNorm = False
num_classes = 3
input_shape = (4,8,8)

### DON'T MODIFY BELOW ###
# Generate dataset variables
fileName = tableBase + '.hdf5'
dataSetName = tableBase + '_onlyLegal'
if not convertStates: 
    dataSetName = tableBase + '_onlyLegal_fullStates'
dataSetWdlName = tableBase + '_Wdl_onlyLegal_3Values'

# Number of Pieces
nPi =  int(dataSetName[0])
nPa = nPi - 2
nWPa = math.ceil(nPa/2)

# Experiment 1
Bengio methood 3n4 with freeze

### Exp 1 Paramters

In [5]:
%run -i 'arena.py'

# Parameters
sourceNet = '103' # trained on 3pc from scratch
# sourceNet = '107' # trained on 4pc from scratch
freeze = True
resSaveFile = '3n4freeze'
epochs = 1
averageOver = 1
expDescr = "Bengio 3n4 - freeze = {} - average over {} runs".format(str(freeze), averageOver)

saveEveryRun = False # save stuff in results dir
saveWeightsCheckpoints = False # save chkp in results dit
saveTensorboardLogs = False # save logs in ./logs dir
resID = '---NORESID---' # used when not saving data, but fitModel() still needs a resID

fractionOfDataToUse = 0.1
plotDuringTraining = True
loadWeights = False 
askForConfirmation = False
saveDir = 'bengioResults'

resSaveFile = resSaveFile + '-{}runAverage'.format(averageOver)
resSaveFileFullPath = saveDir + '/' + str(resSaveFile) + '.pkl' 

### Create model and load data

In [6]:
# prepare save file
# if not os.path.exists(resSaveFileFullPath):
#     print("Save file doesn't exists, creating...\n")
#     save_obj(saveDir, resSaveFile, [])
# else:
#     print("Save file exists...\n")


# load data
X_train, X_test, y_train, y_test = loadData()

# create model
model, nnStr = createModel()
layersCount = len(model.layers)

# load old results
# results = load_obj(saveDir, resSaveFile)

# initialize variables wrt old results
# startTrainingAtLayer = len(results)
startTrainingAtLayer = 0
# print("\nStarting/restarting TL at {} transfered layers".format(startTrainingAtLayer))

X_train shape: (10942, 4, 8, 8)
y_train shape: (10942, 1)
X_test shape: (5390, 4, 8, 8)
y_test shape: (5390, 1)
10942 train samples
5390 test samples
Done loading dataset
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 7, 7)          272       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 6, 32)         928       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 5, 32)         4128      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 4, 64)         8256      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 12, 3, 128)        32896     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 1

### Train

In [54]:
def loadNFirstLayers(model, sourceNet, copyFirstNLayers, freeze):
    # Load weights
#     weightsPath = 'Results/' + sourceNet + '/weights.hdf5'
#     print("Loading first {} layers from results {}, ".format(copyFirstNLayers, weightsPath))
#     model.load_weights(weightsPath)

    # Randomize all but first n layers
    session = K.get_session()
    layers = model.layers
    
    for i in range(len(layers)):
        layer = layers[i]
        
        if hasattr(layer, 'kernel_initializer'):
        
            # freeze layer
            if i < copyFirstNLayers:
                if freeze:
                    print("- {}: Freezing layer {}".format(i+1,layer))
                    layer.trainable = False
                
            # randomize layer
            else:
                print('- {}: Resetting layer {}'.format(i+1,layer))
                layer.kernel.initializer.run(session=session)
                
        else:
            print('- {}: Skipping layer {}'.format(i+1,layer))

    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    return model

In [55]:
# %run -i 'arena.py'
model, nnStr = createModel()
for layer in model.layers:
    print(layer, layer.trainable)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_57 (Conv2D)           (None, 16, 7, 7)          272       
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 15, 6, 32)         928       
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 14, 5, 32)         4128      
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 13, 4, 64)         8256      
_________________________________________________________________
conv2d_61 (Conv2D)           (None, 12, 3, 128)        32896     
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 11, 2, 128)        65664     
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 10, 1, 128)        65664     
__________

In [56]:
compareResultsDuringTraining = False
plotDuringTraining = False
saveTensorboardLogs = False
epochs = 1
model = loadNFirstLayers(model, sourceNet, 9 , freeze)
print(model.summary())
for layer in model.layers:
    print(layer, layer.trainable)

- 1: Freezing layer <keras.layers.convolutional.Conv2D object at 0x7f3d2c10c278>
- 2: Freezing layer <keras.layers.convolutional.Conv2D object at 0x7f3d2c0fb550>
- 3: Freezing layer <keras.layers.convolutional.Conv2D object at 0x7f3d2c086f28>
- 4: Freezing layer <keras.layers.convolutional.Conv2D object at 0x7f3d0c7d8208>
- 5: Freezing layer <keras.layers.convolutional.Conv2D object at 0x7f3d0c7eb588>
- 6: Freezing layer <keras.layers.convolutional.Conv2D object at 0x7f3d0c798208>
- 7: Freezing layer <keras.layers.convolutional.Conv2D object at 0x7f3d0c7ad588>
- 8: Skipping layer <keras.layers.core.Flatten object at 0x7f3d0c742048>
- 9: Freezing layer <keras.layers.core.Dense object at 0x7f3d0c742f28>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_57 (Conv2D)           (None, 16, 7, 7)          272       
_________________________________________________________________
conv2d_58 (Conv2D)       

In [46]:
score = calcScore(model)
fitHistory, logDir = trainModel(resID, model, saveWeightsCheckpoints, saveTensorboardLogs)
score = calcScore(model)

Calculating score
5390/5390 [==============================] - 1s 101us/step
(10942, 4, 8, 8)
Evaluated test loss: 0.46403493478700714
Evaluated test accuracy: 0.8385899814028908
Train on 10942 samples, validate on 5390 samples
Epoch 1/1
10942/10942 [==============================] - 0s 26us/step - loss: 0.5042 - acc: 0.8207 - val_loss: 0.4640 - val_acc: 0.8386
Training done
Calculating score
5390/5390 [==============================] - 0s 55us/step
(10942, 4, 8, 8)
Evaluated test loss: 0.46403493478700714
Evaluated test accuracy: 0.8385899814028908


In [6]:
%run -i 'arena.py'

for copyFirstNLayers in range(startTrainingAtLayer, layersCount):
    print('\n\n')
    print('==========================================================================================')
    print('=                                                                                        =')
    print('=                    Currently transfering first {} layers, out of {}                      ='.format(copyFirstNLayers, layersCount - 1))
    print('=                                                                                        =')
    print('==========================================================================================')
    print()
    
    if copyFirstNLayers == layersCount - 1:
        copyFirstNLayers += 1
        
    accumulatedScore = 0
    for a in range(averageOver):
        # save current averagePosition to tmp file
        with open(saveDir + '/' + str(resSaveFile) + '_currentPosition.txt','w') as file:
            file.write('Currently at transferedLayers = {} out of {} \nInner avg loop position: {} out of {}'.format(copyFirstNLayers, layersCount-1, a+1, averageOver))                     
        # load Model layers        
        model = loadNFirstLayers(model, sourceNet, copyFirstNLayers , freeze)

        # Prepare save dir
        if saveEveryRun:
            resID = genNextResultsDir(model)

        # train
        fitHistory, logDir = trainModel(resID, model, saveWeightsCheckpoints, saveTensorboardLogs)

        # score and save results
        score = calcScore(model)
        if saveEveryRun:
            saveTrainResults(resID, model, logDir, score, copyFirstNLayers)

        # update Return
        accumulatedScore += score[1]
        
    # append averaged results for one set of layers
    results.append(accumulatedScore/averageOver)

    # save old results to checkpoints dir
    dateTime = time.strftime('%Y-%m-%d-%H:%M:%S', time.localtime())
    src = saveDir + '/' + str(resSaveFile) + '.txt'
    dest = saveDir + '/checkpoints/' + str(resSaveFile) + dateTime + '.txt'
    if os.path.exists(src):
        shutil.move(src, dest)

    # save results 
    save_obj(saveDir, resSaveFile, results)
    with open(saveDir + '/' + str(resSaveFile) + '.txt','w') as file:
        file.write(str(results))

    # to load:
    # results = load_obj('temp','3n4.txt')
print('\n Final Results: {}'.format(results))




=                                                                                        =
=                    Currently transfering first 4 layers, out of 8                      =
=                                                                                        =

Loading first 4 layers from results Results/103/weights.hdf5, 
- Resetting layer nr 5: <keras.layers.convolutional.Conv2D object at 0x7f1139a15ef0>
- Resetting layer nr 6: <keras.layers.convolutional.Conv2D object at 0x7f11399c6198>
- Resetting layer nr 7: <keras.layers.convolutional.Conv2D object at 0x7f11399de550>
- Skipping layer nr 8: <keras.layers.core.Flatten object at 0x7f1139977e10>
- Resetting layer nr 9: <keras.layers.core.Dense object at 0x7f1139977da0>


/home/frimann/anaconda3/envs/endnetGpu/lib/python3.5/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Train on 4982178 samples, validate on 2453910 samples
Epoch 1/10
4982178/4982178 [==============================] - 171s 34us/step - loss: 0.2182 - acc: 0.9072 - val_loss: 0.1634 - val_acc: 0.9325
Epoch 2/10
4982178/4982178 [==============================] - 171s 34us/step - loss: 0.1383 - acc: 0.9426 - val_loss: 0.1284 - val_acc: 0.9478
Epoch 3/10
4982178/4982178 [==============================] - 171s 34us/step - loss: 0.1143 - acc: 0.9533 - val_loss: 0.1098 - val_acc: 0.9558
Epoch 4/10
4982178/4982178 [==============================] - 172s 34us/step - loss: 0.1002 - acc: 0.9595 - val_loss: 0.1007 - val_acc: 0.9595
Epoch 5/10
4982178/4982178 [==============================] - 172s 35us/step - loss: 0.0912 - acc: 0.9635 - val_loss: 0.0941 - val_acc: 0.9625
Epoch 6/10
4982178/4982178 [==============================] - 172s 35us/step - loss: 0.0846 - acc: 0.9663 - val_loss: 0.0860 - val_acc: 0.9659
Epoch 7/10
4982178/4982178 [==============================] - 172s 35us/step - loss: 0.0

KeyboardInterrupt: 